In [3]:
import itertools
from math import gcd
from typing import List, Tuple

def get_coefficients(N: int) -> dict:
    '''
    вычисляем коэффициенты
    '''
    mod5 = N % 5
    
    # коэффициенты
    a = [(i + N) % 4 for i in range(9)]
    b = [(j + N) % 7 for j in range(7)]
    c = [(k + N) % 5 for k in range(5)]
    d = [(l + N) % 9 for l in range(4)]
    r = [(m + N) % 11 for m in range(8)]
    s = [(t + N) % 11 for t in range(4)]
    
    # параметры поля
    if mod5 == 0:
        p1, m1 = 5, 3
    elif mod5 == 1:
        p1, m1 = 3, 4
    elif mod5 == 2:
        p1, m1 = 2, 7
    elif mod5 == 3:
        p1, m1 = 13, 2
    else:
        p1, m1 = 11, 2
    
    s2, s1, s0 = s[2], s[1], s[0]
    
    return {
        'a': a, 'b': b, 'c': c, 'd': d, 'r': r, 's': s,
        'p1': p1, 'm1': m1, 's2': s2, 's1': s1, 's0': s0
    }


def task1_roots(N: int) -> dict:
    """Задание 1: Найти корни полиномов"""
    coeffs = get_coefficients(N)
    a, b = coeffs['a'], coeffs['b']
    
    # полином 1
    roots_poly1 = []
    for x in range(4):
        value = (x**9) % 4
        for i in range(9):
            value = (value + a[i] * (x**i)) % 4
        if value % 4 == 0:
            roots_poly1.append(x)
    
    # полином 2
    roots_poly2 = []
    for x in range(7):
        value = 0
        for i in range(7):
            value = (value + b[i] * (x**i)) % 7
        if value % 7 == 0:
            roots_poly2.append(x)
    
    return {
        'poly1_roots': roots_poly1,
        'poly2_roots': roots_poly2,
        'poly1_coeffs': a,
        'poly2_coeffs': b
    }


def task2_factorization(N: int) -> dict:
    """задание 2 исследование приводимости полиномов"""
    coeffs = get_coefficients(N)
    c, d_coeffs = coeffs['c'], coeffs['d']
    
    # полином 1
    def eval_poly1(x, p=5):
        value = (x**5) % p
        for i in range(5):
            value = (value + c[i] * (x**i)) % p
        return value
    
    # проверка на корни
    roots_poly1 = [x for x in range(5) if eval_poly1(x) == 0]
    is_reducible_poly1 = len(roots_poly1) > 0
    
    # упрощёная факторизация полинома
    factors_poly1 = []
    if is_reducible_poly1:
        for root in roots_poly1:
            factors_poly1.append(f"(x - {root})")
    
    # полином 2
    def eval_poly2(x, p=9):
        value = (x**4) % p
        for i in range(4):
            value = (value + d_coeffs[i] * (x**i)) % p
        return value
    
    # проверка на корни для полинома 2
    roots_poly2 = [x for x in range(9) if eval_poly2(x) == 0]
    is_reducible_poly2 = len(roots_poly2) > 0
    
    factors_poly2 = []
    if is_reducible_poly2:
        for root in roots_poly2:
            factors_poly2.append(f"(x - {root})")
    
    return {
        'poly1_reducible': is_reducible_poly1,
        'poly1_factors': factors_poly1,
        'poly2_reducible': is_reducible_poly2,
        'poly2_factors': factors_poly2,
        'poly1_coeffs': c,
        'poly2_coeffs': d_coeffs
    }


def task3_gcd_representation(N: int) -> dict:
    '''
    задание 3 НОД и его линейное представление
    '''
    coeffs = get_coefficients(N)
    r, s_coeffs = coeffs['r'], coeffs['s']
    
    # Полиномы над F11
    def poly_f(x):
        value = 0
        for i in range(8):
            value = (value + r[i] * (x**i)) % 11
        return value % 11
    
    def poly_g(x):
        value = 0
        for i in range(4):
            value = (value + s_coeffs[i] * (x**i)) % 11
        return value % 11
    
    # упрощенный алгоритм Евклида для полиномов
    def poly_gcd(a_coeffs, b_coeffs, p=11):
        a = [c % p for c in a_coeffs]
        b = [c % p for c in b_coeffs]
        
        while any(b):  # пока b не нулевой
            # степени полиномов
            deg_a = len(a) - 1
            deg_b = len(b) - 1
            
            if deg_a < deg_b:
                a, b = b, a
                continue
            
            # коэффициент для деления
            lead_a = a[-1]
            lead_b = b[-1]
            
            # множитель для выравнивания старших коэффициентов
            factor = (lead_a * pow(lead_b, p-2, p)) % p
            
            for i in range(deg_b + 1): # вычитание
                idx = deg_a - deg_b + i
                a[idx] = (a[idx] - factor * b[i]) % p
            
            # убираем ведущие нули
            while len(a) > 0 and a[-1] == 0:
                a.pop()
            if not a:
                a = [0]
        
        return a
    
    gcd_coeffs = poly_gcd(r, s_coeffs)
    
    return {
        'f_coeffs': r,
        'g_coeffs': s_coeffs,
        'gcd_coeffs': gcd_coeffs,
        'gcd_degree': len(gcd_coeffs) - 1 if gcd_coeffs != [0] else -1
    }


def task4_inverse_polynomial(N: int) -> dict:
    '''
    задание 4 обратный полином
    '''
    coeffs = get_coefficients(N)
    s2, s1, s0 = coeffs['s2'], coeffs['s1'], coeffs['s0']
    
    # f(x) = s2*x^2 + s1*x + s0
    f_coeffs = [s0 % 13, s1 % 13, s2 % 13]
    
    # g(x) = x^8 + x^4 + x^3 + 6x + 2
    g_coeffs = [2, 6, 0, 1, 1, 0, 0, 0, 1]  # коэффициенты от x^0 до x^8
    
    # расширенный алгоритм Евклида для нахождения обратного
    def extended_gcd(a, b, p=13):
        if len(b) == 1 and b[0] == 0:
            return a, [1], [0]
        
        # дделение полиномов
        def poly_div(dividend, divisor):
            dividend = dividend.copy()
            divisor = divisor.copy()
            
            deg_dividend = len(dividend) - 1
            deg_divisor = len(divisor) - 1
            
            if deg_dividend < deg_divisor:
                return [0], dividend
            
            quotient = [0] * (deg_dividend - deg_divisor + 1)
            
            for i in range(deg_dividend - deg_divisor, -1, -1):
                if dividend[i + deg_divisor] != 0:
                    factor = (dividend[i + deg_divisor] * pow(divisor[deg_divisor], p-2, p)) % p
                    quotient[i] = factor
                    
                    for j in range(deg_divisor + 1):
                        dividend[i + j] = (dividend[i + j] - factor * divisor[j]) % p
            
            # убираем ведущие нули
            while len(dividend) > 0 and dividend[-1] == 0:
                dividend.pop()
            if not dividend:
                dividend = [0]
            
            return quotient, dividend
        
        quot, rem = poly_div(a, b)
        g, u, v = extended_gcd(b, rem, p)
        
        new_u = v
        new_v = [0] * max(len(u), len(quot))
        for i in range(len(u)):
            new_v[i] = (new_v[i] + u[i]) % p
        for i in range(len(quot)):
            if i < len(v):
                new_v[i] = (new_v[i] - quot[i] * v[i]) % p
            else:
                new_v.append((-quot[i] * v[i]) % p)
        
        return g, new_u, new_v
    
    gcd_coeffs, u_coeffs, v_coeffs = extended_gcd(g_coeffs, f_coeffs, 13)
    
    # Если НОД = 1, в u_coeffs содержится обратный
    if len(gcd_coeffs) == 1 and gcd_coeffs[0] == 1:
        inverse_coeffs = u_coeffs
    else:
        inverse_coeffs = []
    
    return {
        'f_coeffs': f_coeffs,
        'g_coeffs': g_coeffs,
        'inverse_coeffs': inverse_coeffs,
        'has_inverse': len(inverse_coeffs) > 0
    }


def generate_irreducible_polynomials(q: int, d: int) -> list:
    """
    Возвращает список всех неприводимых полиномов степени d над F_q.
    """
    def poly_mod(a, b, q):
        ''' 
        деление полиномов по модулю q с остатком
        '''
        a = list(a)
        b = list(b)
        
        while len(a) >= len(b) and len(a) > 0 and a[-1] != 0:
            if len(b) == 0 or b[-1] == 0:
                break
                
            # коэффициент для деления
            factor = (a[-1] * pow(b[-1], q-2, q)) % q
            
            # вычитаем
            shift = len(a) - len(b)
            for i in range(len(b)):
                a[i + shift] = (a[i + shift] - factor * b[i]) % q
            
            # убираем ведущие нули
            while len(a) > 0 and a[-1] == 0:
                a.pop()
        
        return a
    
    def poly_gcd(a, b, q):
        '''
        НОД полиномов
        '''
        a, b = list(a), list(b)
        while b and any(x != 0 for x in b):
            a, b = b, poly_mod(a, b, q)
        return a
    
    def is_irreducible(poly_coeffs, q):
        '''
        проверяет неприводимость полинома с помощью алгоритма Берлекэмпа 
        (каюсь, не моя идея с этим алгоритмом, ну а что поделать,
        я не знал как проверить неприводимость)
        '''
        n = len(poly_coeffs) - 1
        
        # проверка на наличие корней в поле F_q
        for x in range(q):
            value = 0
            for i in range(n + 1):
                value = (value + poly_coeffs[i] * pow(x, i, q)) % q
            if value == 0:
                return False
        
        # проверка делителей степени меньше n/2
        for k in range(1, n // 2 + 1):
            # генерируем x^(q^k) - x mod poly
            x_poly = [0] * (q**k + 1)
            x_poly[1] = 1  # x
            x_qk_poly = [1]  # x^(q^k)
            
            # вычисляем x^(q^k) mod poly методом повторного возведения в квадрат
            for _ in range(k):
                new_poly = [0] * (2 * len(x_qk_poly) - 1)
                for i in range(len(x_qk_poly)):
                    for j in range(len(x_qk_poly)):
                        new_poly[i + j] = (new_poly[i + j] + x_qk_poly[i] * x_qk_poly[j]) % q
                x_qk_poly = poly_mod(new_poly, poly_coeffs, q)
            
            # вычисляем x^(q^k) - x
            x_qk_minus_x = x_qk_poly.copy()
            if len(x_qk_minus_x) <= 1:
                x_qk_minus_x.extend([0] * (2 - len(x_qk_minus_x)))
            x_qk_minus_x[1] = (x_qk_minus_x[1] - 1) % q
            
            # проверяем НОД(poly, x^(q^k) - x)
            gcd_result = poly_gcd(poly_coeffs, x_qk_minus_x, q)
            if len(gcd_result) > 1 or gcd_result[0] != 1:
                return False
        
        return True
    
    irreducible_polys = []
    
    # генерируем все полиномы степени d с ненулевым старшим коэффициентом
    for coeffs in itertools.product(range(q), repeat=d + 1):
        if coeffs[-1] != 0:  # старший коэффициент не нуль
            poly_list = list(coeffs)
            if is_irreducible(poly_list, q):
                irreducible_polys.append(poly_list)
    
    return irreducible_polys


